In [3]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [4]:
def preprocessing(document): #특수문자 기본 전처리 함수(토큰화할때 마침표제거, 숫자변환)
    docu = []
    for doc in document:
        doc = re.sub('[^가-힣0-9\s.]','',doc)
        doc = re.sub('[-=+,.#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',doc)
#         doc = re.sub('[0-9]+','num',doc) #숫자를 special token num으로 치환
        pattern = re.compile(r'\s+') #중복띄어쓰기 제거
        doc = re.sub(pattern,' ',doc)
        doc = doc.replace('\xa0','') 
        doc = doc.replace('\n','')
        docu.append(doc)

    return docu

In [5]:
def final_url(url): #사업보고서 주소를 넣으면 '이사의 경영진단 및 분석' 주소 반환
    try:
        req2 = requests.get(url)
        html2 = req2.text
        soup2 = BeautifulSoup(html2, 'html.parser')
        body = str(soup2.find('head'))

        a = re.search('이사의 경영진단 및 분석의견',body).span()
        b= re.search(r'viewDoc(.*);',body[a[0]:]).group()
        ls = b[8:-2].split(',')
        ls = [i[1:-1] for i in ls]
        ls[1] = ls[1][1:] #드러움
        ls[2] = ls[2][1:]
        ls[3] = ls[3][1:]
        ls[4] = ls[4][1:]
        ls[5] = ls[5][1:]    
        
        url_final = 'http://dart.fss.or.kr/report/viewer.do?rcpNo='+ ls[0] + '&dcmNo='+ ls[1]+'&eleId=' +ls[2] + '&offset=' + ls[3] + '&length=' + ls[4] + '&dtd=dart3.xsd'
        return(url_final)
    
    except AttributeError as e:
        print(e)

In [6]:
def extracting_text(url): #경영진단 url 넣었을 시 해당 본문 텍스트 반환
    req3 = requests.get(url)
    html3 = req3.text
    soup3 = BeautifulSoup(html3,'html.parser')
    tables = []
    ka = soup3.find_all('span') #table을 제외한 본문 text 부분 찾아서 리스트안에저장
    for k in ka:
        tables.append(k.get_text())
    table = ''.join(tables) #리스트를 string으로 바꾼다
    table = table.replace('\xa0','') #정리
    table = table.replace('\n','')
    # table = re.sub('[[a-zA-Z]',"",table) #전처리
    # table = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',table)
    #table = re.sub('[0-9]+','num',table) #숫자를 special token num으로 치환
    # pattern = re.compile(r'\s+') #중복띄어쓰기 제거
    # table = re.sub(pattern,' ',table)
    return table

In [7]:
def extracting_text2(url): #경영진단 url 넣었을 시 해당 본문 텍스트 반환
    req3 = requests.get(url)
    html3 = req3.text
    soup3 = BeautifulSoup(html3,'html.parser')
    tables = []
    ka = soup3.find_all('p') #table을 제외한 본문 text 부분 찾아서 리스트안에저장
    for k in ka:
        tables.append(k.get_text())
    table = ''.join(tables) #리스트를 string으로 바꾼다
    table = table.replace('\xa0','') #정리
    table = table.replace('\n','')
    # table = re.sub('[[a-zA-Z]',"",table) #전처리
    # table = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',table)
    #table = re.sub('[0-9]+','num',table) #숫자를 special token num으로 치환
    # pattern = re.compile(r'\s+') #중복띄어쓰기 제거
    # table = re.sub(pattern,' ',table)
    return table

In [8]:
df_all = pd.read_csv('./data/crawling//preprocessed/LV1_kospi200_all.csv',index_col=0)

In [9]:
k = pd.DataFrame({'crp_nm':['SK하이닉스','LG','LG','LS','LS','LS','LS산전','LS산전','LS산전','삼성증권'
                            ,'삼성카드','삼성카드','삼성카드'],
                  'crp_cd':[660,3550,3550,6260,6260,6260,10120,10120,10120,16360,29780,29780,29780],
                  '연도' : [2018,2018,2017,2013,2015,2017,2015,2016,2017,2018,2016,2017,2018],
                      'rpt_nm': ['사업보고서 (2018.12)','사업보고서 (2018.12)','사업보고서 (2017.12)','사업보고서 (2013.12)','사업보고서 (2015.12)',
              '사업보고서 (2017.12)','사업보고서 (2015.12)','사업보고서 (2016.12)','사업보고서 (2017.12)',
              '사업보고서 (2018.12)','사업보고서 (2016.12)','사업보고서 (2017.12)','사업보고서 (2018.12)'],
                  'rcp_no':[20190404002329,20190401004643,20180402005213,20140515000682,20160330002783,20180402005195,20160330004015,20170331003670
,20180330003041,20190401005163,20170331005195,20180402003595,20190401003649],
                'rcp_dt':[20190404,20190401,20180402,20140515,20160330,20180402,20160330,20170331,20180330,20190401,20170331,20180402,20190401],
                 'rmk' : ['연','연','연','연','연','연','연','연','연','연','연','연','연']
                 })



크롤링오류인지모르겠지만 안뽑힌애들

SK하이닉스2018 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190404002329
LG 2018 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401004643
LG 2017 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402005213
LS 2013 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20140515000682
LS 2015 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330002783
LS 2017 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402005195
LS산전 2015 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330004015
LS산전 2016 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331003670
LS산전 2017 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180330003041
삼성증권 2009 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20090629000546
삼성증권 2013 index 705,706 중복인지 알았는데 그게아니라 2013이름으로 두번올라온것 실제로는 rcp_dt 기준으로 '연도'생성 문제없음
삼성증권 2019 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401005163
삼성카드 2017 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331005195
삼성카드 2018 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402003595
삼성카드 2019 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401003649

In [10]:
urls = ['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190404002329'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401004643'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402005213'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20140515000682'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330002783'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402005195'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330004015'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331003670'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180330003041'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401005163'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331005195'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402003595'
 ,'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401003649']

In [11]:
f_urls = [final_url(x) for x in urls]

In [12]:
ex_str_span = [extracting_text2(x) for x in f_urls]

In [13]:
k['str'] = ex_str_span

In [14]:
df_new = pd.concat([df_all,k]).reset_index(drop=True)

In [15]:
unique_ls = [x for x in df_all.crp_nm.unique()]
for i in unique_ls:
    if len(df_all[df_all.crp_nm == i]) != 10:
        print(i)


SK하이닉스
LG
LS
LS산전
삼성증권
삼성카드


In [16]:
unique_ls = [x for x in df_new.crp_nm.unique()]
fe = []
for i in unique_ls:
    if len(df_new[df_new.crp_nm == i]) == 10:
        fe.append(i)
print(len(fe))


113


연도 column 에 대해서 명확히 하고 넘어가자면 rcp_dt 와 맞춰줘야 하는것 같다.
2018년에 대한 사업보고서는 2019년에 나오지만 발행일자 기준으로 주가를 살펴볼 것이기 때문이다.
2010과 2019년 사이의 10년치 데이터는  2009~2018년 회사 상황에 대한 내용을 정리한 것이다.

In [17]:
df_new = df_new.drop('연도',axis=1)

In [18]:
df_new.insert(0,'연도',in_ls)

NameError: name 'in_ls' is not defined

In [ ]:
em_ls = [str(x)[:4] for x in df_new.rcp_dt]
in_ls = [int(x) for x in em_ls]

In [ ]:
df_new

In [ ]:
df_new.str = preprocessing(df_new.str)#새로추출된 텍스트들에 대해 전처리실행

In [ ]:
df_new

삼성증권이 2013.03 / 2013.12 란 제목으로 공시를 두 번 하였는데 이게 실제 사업보고서 날짜상으로 매칭시키면 
그 이전에는 6월에 올라오다가 2013년부터 다른기업들처럼 3월에 사업보고서를 제출하는걸로 바뀌어서 rcp_dt 기준으로 '연도' column을 만들었고 실제 텍스트 상의 내용도 다르지 않기에 제외하지 않고 이름만 바꾸어주었다.

'sk 하이닉스' 공시문서들에 있어서 오류가 있어서 다시 크롤링해야한다. 단어수로 sorting했을 때 굉장히 짧게 나와서.. 

사업보고서 (2009~2014.12) 까지 6개 다시 크롤링해야된다. 텍스트 인덱스는 83~88까지

In [24]:
df_new[df_new.crp_nm == 'SK하이닉스']

,crp_nm,crp_cd,rpt_nm,rcp_no,rcp_dt,rmk,str
80,SK하이닉스,660,사업보고서 (2017.12),20180402004745,20180402,정연,. 이사의 경영진단 및 분석의견예측정보에 대한 주의사항본 사업보고서의 이사의 경영진...
81,SK하이닉스,660,사업보고서 (2016.12),20170331004537,20170331,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에 ...
82,SK하이닉스,660,사업보고서 (2015.12),20160330003963,20160330,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에 ...
83,SK하이닉스,660,사업보고서 (2014.12),20150331004519,20150331,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항2. 경영성과3. 재무상...
84,SK하이닉스,660,[첨부추가]사업보고서 (2013.12),20140331002980,20140331,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항2. 경영성과3. 재무상...
85,SK하이닉스,660,사업보고서 (2012.12),20130329002163,20130329,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항2. 경영성과3. 재무상...
86,SK하이닉스,660,사업보고서 (2011.12),20120330001870,20120330,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항2. 경영성과3. 재무상...
87,SK하이닉스,660,사업보고서 (2010.12),20110331003040,20110331,연,. 이사의 경영진단 및 분석의견1. 이사의 경영진단 의견서가. 재무상태 및 영업실적...
88,SK하이닉스,660,사업보고서 (2009.12),20100331000696,20100331,연,. 이사의 경영진단 및 분석의견1. 이사의 경영진단 의견서가. 재무상태 및 영업실적...
1117,SK하이닉스,660,사업보고서 (2018.12),20190404002329,20190404,연,IV. 이사의 경영진단 및 분석의견예측정보에 대한 주의사항본 사업보고서의 '이사의 ...


In [29]:
urls = ['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20100331000696',
        'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331003040',
        'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20120330001870',
        'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20130329002163',
        'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20140331002980',
        'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20150331004519'
]

In [30]:
ex_urls = [final_url(x) for x in urls]

In [31]:
ex_urls

['http://dart.fss.or.kr/report/viewer.do?rcpNo=20100331000696&dcmNo=2586641&eleId=13&offset=459497&length=17932&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20110331003040&dcmNo=2970028&eleId=13&offset=454654&length=19774&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20120330001870&dcmNo=3330901&eleId=13&offset=481499&length=21656&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20130329002163&dcmNo=3699046&eleId=13&offset=391822&length=30353&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20140331002980&dcmNo=4117770&eleId=13&offset=351000&length=27468&dtd=dart3.xsd',
 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20150331004519&dcmNo=4564311&eleId=19&offset=1361072&length=30746&dtd=dart3.xsd']

In [32]:
sk_text = [extracting_text2(x) for x in ex_urls]

In [46]:
df_new[df_new.crp_nm == 'SK하이닉스']

,crp_nm,crp_cd,rpt_nm,rcp_no,rcp_dt,rmk,str
80,SK하이닉스,660,사업보고서 (2017.12),20180402004745,20180402,정연,. 이사의 경영진단 및 분석의견예측정보에 대한 주의사항본 사업보고서의 이사의 경영진...
81,SK하이닉스,660,사업보고서 (2016.12),20170331004537,20170331,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에 ...
82,SK하이닉스,660,사업보고서 (2015.12),20160330003963,20160330,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에 ...
83,SK하이닉스,660,사업보고서 (2014.12),20150331004519,20150331,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항2. 경영성과3. 재무상...
84,SK하이닉스,660,[첨부추가]사업보고서 (2013.12),20140331002980,20140331,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항2. 경영성과3. 재무상...
85,SK하이닉스,660,사업보고서 (2012.12),20130329002163,20130329,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항2. 경영성과3. 재무상...
86,SK하이닉스,660,사업보고서 (2011.12),20120330001870,20120330,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항2. 경영성과3. 재무상...
87,SK하이닉스,660,사업보고서 (2010.12),20110331003040,20110331,연,. 이사의 경영진단 및 분석의견1. 이사의 경영진단 의견서가. 재무상태 및 영업실적...
88,SK하이닉스,660,사업보고서 (2009.12),20100331000696,20100331,연,. 이사의 경영진단 및 분석의견1. 이사의 경영진단 의견서가. 재무상태 및 영업실적...
1117,SK하이닉스,660,사업보고서 (2018.12),20190404002329,20190404,연,IV. 이사의 경영진단 및 분석의견예측정보에 대한 주의사항본 사업보고서의 '이사의 ...


In [43]:
sk_text[0]

"V. 이사의 경영진단 및 분석의견1. 이사의 경영진단 의견서가. 재무상태 및 영업실적1. 매출2009년 당사 총 매출액은 2008년 6조 4,953억원에 비해 약 16%, 1조 261억원 증가한 7조5,214억원을 기록하였습니다. 이와 같이 매출이 증가한 이유는 전반적으로 원·달러 환율상승으로 매출증가요인이 있었던 가운데, 제한된 공급 속에 세계 경기 회복과 맞물려 메모리반도체 시장의 수요가 빠르게 회복되며 판매가격이 예년에 비해 견조한 흐름을 보인 것이 주된 원인입니다.세부적으로는 DRAM의 경우 판매가격의 하락을 상회하는 매출수량 증가로 매출이 증가하였으며, 매출비중도 2008년 73%에서 소폭 증가한 76%를 차지하였습니다. 한편 NAND Flash의 경우에는 연간 판매가격이 하락한 가운데 200mm 조업 중단으로 인한 매출수량 감소로 매출이 소폭 감소하였으며, NAND Flash 및 기타 매출 비중도 2008년 21%에서 17%로 감소하였습니다.2. 매출원가2009년 당사의 매출원가는 2008년의 7조 4,822억원에 비해 약 15%, 1조 1,454억원 감소한 6조 3,368억원을 기록하였습니다. 매출원가의 감소요인은, 판매가격의 급격한 하락으로 2008년의 원가증가 원인이 되었던 재고자산평가손실금액의 상당부분이 2009년에는 판매가격이 회복되어 재고자산평가손실 환입으로 반영되었기 때문입니다.3. 판관비2009년 당사의 판매관리비는 2008년의 1조 2,154억원에 비해 약 7%, 851억원 증가한 1조 3,005억원을 기록하였습니다. 이는 주로 경상기술료 등의 지급수수료 증가 및 신제품또는 신기술의 연구 및 개발활동과 관련된 경상개발비가 증가하였기 때문입니다.한편 영업상의 분쟁 및 특허관련 소송 등으로 인한 지급수수료가 744억원 증가하였습니다.4. 영업외수익2009년 당사의 영업외수익은 2008년의 8,783억원에 비해 약 42%, 3,693억원 증가한 1조2,476억원을 기록하였습니다. 이는 환율변동에 따른 환율효과로 외환차익 및 외화환산이익이

## SK 하이닉스 애들 잘안뽑힌 것 뽑았으니 원 데이터프레임에 추가를 하자

In [51]:
df_new = pd.read_csv('./data/crawling/preprocessed/df_new.csv',index_col=0)

In [53]:
df_new[df_new.crp_nm == 'SK하이닉스']

,연도,crp_nm,crp_cd,rpt_nm,rcp_no,rcp_dt,rmk,str
80,2018,SK하이닉스,660,사업보고서 (2017.12),20180402004745,20180402,정연,이사의 경영진단 및 분석의견예측정보에 대한 주의사항본 사업보고서의 이사의 경영진단...
81,2017,SK하이닉스,660,사업보고서 (2016.12),20170331004537,20170331,연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항당사가 동 사업보고서에 기재...
82,2016,SK하이닉스,660,사업보고서 (2015.12),20160330003963,20160330,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항당사가 동 사업보고서에 기재...
83,2015,SK하이닉스,660,사업보고서 (2014.12),20150331004519,20150331,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과3 재무상태 및 ...
84,2014,SK하이닉스,660,[첨부추가]사업보고서 (2013.12),20140331002980,20140331,연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과3 재무상태 및 ...
85,2013,SK하이닉스,660,사업보고서 (2012.12),20130329002163,20130329,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과3 재무상태 및 ...
86,2012,SK하이닉스,660,사업보고서 (2011.12),20120330001870,20120330,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과3 재무상태 및 ...
87,2011,SK하이닉스,660,사업보고서 (2010.12),20110331003040,20110331,연,이사의 경영진단 및 분석의견1 이사의 경영진단 의견서가 재무상태 및 영업실적 나 ...
88,2010,SK하이닉스,660,사업보고서 (2009.12),20100331000696,20100331,연,이사의 경영진단 및 분석의견1 이사의 경영진단 의견서가 재무상태 및 영업실적나 유...
1117,2019,SK하이닉스,660,사업보고서 (2018.12),20190404002329,20190404,연,이사의 경영진단 및 분석의견예측정보에 대한 주의사항본 사업보고서의 이사의 경영진단...


83~85까지 애들 텍스트 바꿔주면된다

In [69]:
df_new.iloc[83:89,:] #뽑은 데이터순서랑 반대로라 list.reverse()했음

,연도,crp_nm,crp_cd,rpt_nm,rcp_no,rcp_dt,rmk,str
83,2015,SK하이닉스,660,사업보고서 (2014.12),20150331004519,20150331,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과3 재무상태 및 ...
84,2014,SK하이닉스,660,[첨부추가]사업보고서 (2013.12),20140331002980,20140331,연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과3 재무상태 및 ...
85,2013,SK하이닉스,660,사업보고서 (2012.12),20130329002163,20130329,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과3 재무상태 및 ...
86,2012,SK하이닉스,660,사업보고서 (2011.12),20120330001870,20120330,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과3 재무상태 및 ...
87,2011,SK하이닉스,660,사업보고서 (2010.12),20110331003040,20110331,연,이사의 경영진단 및 분석의견1 이사의 경영진단 의견서가 재무상태 및 영업실적 나 ...
88,2010,SK하이닉스,660,사업보고서 (2009.12),20100331000696,20100331,연,이사의 경영진단 및 분석의견1 이사의 경영진단 의견서가 재무상태 및 영업실적나 유...


In [71]:
df_new.iloc[83:89,:].str = sk_text #바꿔치기

C:\Users\yphy0\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [79]:
df_new.str = preprocessing(df_new.str) #기본전처리 다시

끝... 이지만 table 태그는 아직 해결못함

In [89]:
df_new[df_new.crp_nm =='SK하이닉스']

,연도,crp_nm,crp_cd,rpt_nm,rcp_no,rcp_dt,rmk,str
80,2018,SK하이닉스,660,사업보고서 (2017.12),20180402004745,20180402,정연,이사의 경영진단 및 분석의견예측정보에 대한 주의사항본 사업보고서의 이사의 경영진단...
81,2017,SK하이닉스,660,사업보고서 (2016.12),20170331004537,20170331,연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항당사가 동 사업보고서에 기재...
82,2016,SK하이닉스,660,사업보고서 (2015.12),20160330003963,20160330,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항당사가 동 사업보고서에 기재...
83,2015,SK하이닉스,660,사업보고서 (2014.12),20150331004519,20150331,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과3 재무상태 및 ...
84,2014,SK하이닉스,660,[첨부추가]사업보고서 (2013.12),20140331002980,20140331,연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과2013년은 미국...
85,2013,SK하이닉스,660,사업보고서 (2012.12),20130329002163,20130329,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과2012년 유럽발...
86,2012,SK하이닉스,660,사업보고서 (2011.12),20120330001870,20120330,정연,이사의 경영진단 및 분석의견1 예측정보에 대한 주의사항2 경영성과2011년 하이닉...
87,2011,SK하이닉스,660,사업보고서 (2010.12),20110331003040,20110331,연,이사의 경영진단 및 분석의견1 이사의 경영진단 의견서가 재무상태 및 영업실적 1 ...
88,2010,SK하이닉스,660,사업보고서 (2009.12),20100331000696,20100331,연,이사의 경영진단 및 분석의견1 이사의 경영진단 의견서가 재무상태 및 영업실적1 매...
1117,2019,SK하이닉스,660,사업보고서 (2018.12),20190404002329,20190404,연,이사의 경영진단 및 분석의견예측정보에 대한 주의사항본 사업보고서의 이사의 경영진단...


In [90]:
df_new.to_csv('./data/crawling/preprocessed/df_new.csv')